In [ ]:
# 📌 Installer et importer les bibliothèques nécessaires
!pip install transformers accelerate  


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [17]:
import os
import json
import torch
import pandas as pd
from glob import glob
from tqdm import tqdm
from transformers import pipeline

In [18]:
# 📌 Charger le modèle Flan-T5
model_name = "google/flan-t5-small"
llm = pipeline("text2text-generation", model=model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [19]:
# Get Data
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


Mounted at /content/drive


In [20]:
# 📌 Charger les fichiers JSON
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 📌 Définir le chemin vers le dataset MedDec (adapté pour Google Drive si nécessaire)
mimic_path = "/content/drive/Shareddrives/AI4H_Drive/01 AI4Health Project/05 Generated File/meddec-mimic-iii"
data_dir = os.path.join(mimic_path, "data")
files = glob(os.path.join(data_dir, "*.json"))

# 📌 Charger et fusionner les JSONs dans un DataFrame
data_list = [load_json(file) for file in tqdm(files, desc="Loading JSON files")]
df = pd.DataFrame(data_list)

# 📌 Afficher un aperçu des données
print("\n🔍 Aperçu des données :", df.head())

Loading JSON files: 100%|██████████| 24/24 [00:00<00:00, 241.05it/s]


🔍 Aperçu des données :   annotator_id discharge_summary_id  \
0       CALIML    11216_188044_9756   
1       CALIML   10814_119849_52793   
2       CALIML    11043_171810_1714   
3       CALIML   10814_180749_52780   
4       CALIML    11043_138702_1715   

                                         annotations  
0  [{'decision': 'This is a 67-year-old male stat...  
1  [{'decision': 'Bacteremia/ Bleeding from G tub...  
2  [{'decision': 'Mr. [**Known firstname 9818**] ...  
3  [{'decision': 'Bleeding from colostomy and fol...  
4  [{'decision': 'Fever/Chills', 'category': 'Cat...  


In [47]:
def create_zero_shot_prompt(text):
    """
    Génère un prompt amélioré pour extraire les décisions médicales en mode Zero-Shot.
    """
    prompt = f"""
    You are a specialized medical assistant helping to extract key medical decisions from clinical discharge summaries.
    Your task is to read the following clinical note and extract clear, structured medical decisions made for the patient.

    For each decision, determine the most appropriate category from the following:
    - Defining problem (e.g., symptoms, diagnosis)
    - Drug related (e.g., prescribed medications, dosage)
    - Evaluating test result (e.g., lab results, imaging findings)
    - Therapeutic procedure related (e.g., surgeries, medical interventions)
    - Contact related (e.g., follow-up appointments, referrals)
    - Advice and precaution (e.g., lifestyle recommendations, risk factors)

    Output Format:
    Return your response as a structured JSON array, where each decision is represented as:
    [
        {{"decision": "<specific medical decision>", "category": "<chosen category>"}}
    ]

    Carefully analyze the clinical note and extract only **clear and relevant** medical decisions.

    Clinical Note: {text}
    """
    return prompt



def create_one_shot_prompt(text):
    """
    Génère un prompt amélioré pour One-Shot avec un exemple détaillé.
    """
    example_input = """
    Clinical Note:
    The patient arrived with symptoms of fever and cough. A chest X-ray was performed and revealed mild pneumonia.
    The patient was prescribed a 7-day course of antibiotics and advised to rest.

    Expected Output:
    [
        {"decision": "fever and cough", "category": "Defining problem"},
        {"decision": "chest X-ray revealed mild pneumonia", "category": "Evaluating test result"},
        {"decision": "7-day course of antibiotics", "category": "Drug related"},
        {"decision": "advised to rest", "category": "Advice and precaution"},
        {"decision": "severe cough and fatigue", "category": "Defining problem"},
        {"decision": "10-day course of antibiotics", "category": "Drug related"},
    ]
    """

    prompt = f"""
    You are a medical assistant trained in analyzing clinical notes.
    Your task is to extract key medical decisions and classify them into one of the following categories:
    - Defining problem (e.g., symptoms, diagnosis)
    - Drug related (e.g., prescribed medications, dosage)
    - Evaluating test result (e.g., lab results, imaging findings)
    - Therapeutic procedure related (e.g., surgeries, medical interventions)
    - Contact related (e.g., follow-up appointments, referrals)
    - Advice and precaution (e.g., lifestyle recommendations, risk factors)

    Follow these rules:
    - Return a structured JSON array.
    - Each decision must have **both a "decision" and a "category"**.
    - Only extract **clear, relevant** medical decisions.

    Here is an example of how the decisions should be structured:
    {example_input}

    Now, analyze the following clinical note and return the medical decisions in the same structured format:

    Clinical Note: {text}
    """
    return prompt


sample_text = df.sample(n=1, random_state=42)["annotations"].values[0][0]["decision"]

print("\n🔹 Aperçu du Zero-Shot Prompt:")
print(create_zero_shot_prompt(sample_text))

print("\n🔹 Aperçu du One-Shot Prompt:")
print(create_one_shot_prompt(sample_text))



🔹 Aperçu du Zero-Shot Prompt:

    You are a specialized medical assistant helping to extract key medical decisions from clinical discharge summaries.
    Your task is to read the following clinical note and extract clear, structured medical decisions made for the patient.

    For each decision, determine the most appropriate category from the following:
    - Defining problem (e.g., symptoms, diagnosis)
    - Drug related (e.g., prescribed medications, dosage)
    - Evaluating test result (e.g., lab results, imaging findings)
    - Therapeutic procedure related (e.g., surgeries, medical interventions)
    - Contact related (e.g., follow-up appointments, referrals)
    - Advice and precaution (e.g., lifestyle recommendations, risk factors)

    Output Format:
    Return your response as a structured JSON array, where each decision is represented as:
    [
        {"decision": "<specific medical decision>", "category": "<chosen category>"}
    ]

    Carefully analyze the clinical not

In [48]:
def evaluate_model(df, sample_size=20, mode="zero-shot"):
    """
    Évalue le modèle sur un échantillon de textes avec un prompt Zero-Shot ou One-Shot.
    """
    df_sample = df.sample(n=sample_size, random_state=42)  # Sélectionne un sous-ensemble aléatoire
    predictions = []

    for i, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc=f"Evaluating {mode} prompt"):
        text = row.get("annotations", [{}])[0].get("decision", "")  # Extraction du texte

        if mode == "zero-shot":
            prompt = create_zero_shot_prompt(text)
        elif mode == "one-shot":
            prompt = create_one_shot_prompt(text)
        else:
            raise ValueError("Mode must be 'zero-shot' or 'one-shot'.")

        response = llm(prompt, max_length=300)  # Allongement du texte pour des réponses plus complètes
        prediction = response[0]["generated_text"]

        predictions.append({
            "summary_excerpt": text[:100],  # Affichage d'un extrait au lieu de l'ID
            "mode": mode,
            "prediction": prediction
        })

    return pd.DataFrame(predictions)

# 📌 Exécuter l’évaluation Zero-Shot et One-Shot
df_zero_shot = evaluate_model(df, sample_size=20, mode="zero-shot")
df_one_shot = evaluate_model(df, sample_size=20, mode="one-shot")

Evaluating one-shot prompt: 100%|██████████| 20/20 [00:56<00:00,  2.82s/it]


In [49]:
# 📌 Afficher les résultats
print("\n📝 Zero-Shot Results:\n", df_zero_shot)
print("\n📝 One-Shot Results:\n", df_one_shot)


📝 Zero-Shot Results:
                                       summary_excerpt       mode  \
0                                     Left Hemothorax  zero-shot   
1       HIV/AIDS, last CD4 count 4, blind [**1-29**]   zero-shot   
2   This is a 67-year-old male status post coronar...  zero-shot   
3                     Klebsiella bacteremia/pneumonia  zero-shot   
4                                         Trisomy 21   zero-shot   
5          Mitral regurgitation/mitral valve prolapse  zero-shot   
6   polycythemia [**Doctor First Name **] for twel...  zero-shot   
7               Bacteremia/ Bleeding from G tube site  zero-shot   
8   delivered at 35 and 1/7 weeks gestation with a...  zero-shot   
9                   Respiratory Distress, Hypotension  zero-shot   
10  Mr. [**Known firstname 9818**] [**Known lastna...  zero-shot   
11                                           Gi bleed  zero-shot   
12                  Diarrhea with fever & tachycardia  zero-shot   
13                  Bleed